# Week 3 Assignment: PART 1

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
!conda install -c anaconda beautifulsoup4 --yes

In [3]:
#!conda install -c anaconda lxml --yes

In [ ]:
!conda install -c anaconda requests --yes

In [5]:
import requests
from bs4 import BeautifulSoup # The Beautiful Soup package is used to extract data from html files.

In [6]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text #text removes the html tags within the url

##### The Beautiful Soup package is used to parse the html -  take the raw html text and break it into Python objects. The second argument 'lxml' or features = 'html' is the html parser.

In [7]:
soup= BeautifulSoup(source,features='html')
type(soup)

bs4.BeautifulSoup

#### Extract the table, rows and cell info from table 1 in wikipedia link.

##### Table: 

In [8]:
table = soup.find_all('table')[0] 
#print(table.prettify())

##### Table Rows: 

In [ ]:
rows= soup.table.find_all('tr') # to print out table rows only
print(rows[:10]) 

##### Table Cells:

In [10]:
#loop that iterates through table rows and prints out the cells of the rows.
for row in rows: 
    row_td= row.find_all('td')
    
print(row_td)
type (row_td)

[<td>M9Z</td>, <td>Not assigned</td>, <td>Not assigned
</td>]


bs4.element.ResultSet

#### Creating a dataframe: 

In [11]:
#- Get all table cells in list form first and then convert that list into a df

import re # import general expressions module and create table

list_rows = []
for row in rows:
    cells = row.find_all('td')
    str_cells = str(cells)
    clean = re.compile('<.*?>') #The .*? will match an opening angle bracket followed by anything and followed by a closing angle bracket.
    clean2 = (re.sub(clean, '',str_cells))
    list_rows.append(clean2)

df = pd.DataFrame(list_rows) # create dataframe from table rows
df.tail(10) # veiw last 10 rows of dataframe

,0
278,"[M4Z, Not assigned, Not assigned\n]"
279,"[M5Z, Not assigned, Not assigned\n]"
280,"[M6Z, Not assigned, Not assigned\n]"
281,"[M7Z, Not assigned, Not assigned\n]"
282,"[M8Z, Etobicoke, Kingsway Park South West\n]"
283,"[M8Z, Etobicoke, Mimico NW\n]"
284,"[M8Z, Etobicoke, The Queensway West\n]"
285,"[M8Z, Etobicoke, Royal York South West\n]"
286,"[M8Z, Etobicoke, South of Bloor\n]"
287,"[M9Z, Not assigned, Not assigned\n]"


#### Cleaning DataFrame: 

In [12]:
#split O column into multiple columns (seperated by a ,)
df1 = df[0].str.split(',', expand=True) 
df1.head(10)

,0,1,2
0,[],None,None
1,[M1A,Not assigned,Not assigned\n]
2,[M2A,Not assigned,Not assigned\n]
3,[M3A,North York,Parkwoods\n]
4,[M4A,North York,Victoria Village\n]
5,[M5A,Downtown Toronto,Harbourfront\n]
6,[M6A,North York,Lawrence Heights\n]
7,[M6A,North York,Lawrence Manor\n]
8,[M7A,Downtown Toronto,Queen's Park\n]
9,[M8A,Not assigned,Not assigned\n]


In [13]:
df1[2] = df1[2].str.strip('\n]') #remove brackets from column 2
df1[0] = df1[0].str.strip('[')  #remove brackets from column 0
df2 = df1.iloc[:, 0:3] #isolate 1st 3 columns of dataframe
df2.head(10)

,0,1,2
0,],None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Downtown Toronto,Queen's Park
9,M8A,Not assigned,Not assigned


In [14]:
#Add headers

col_labels = soup.find_all('th')
all_header = []
col_str = str(col_labels)
cleantext2 = BeautifulSoup(col_str, features='html').get_text()
all_header.append(cleantext2)
print(all_header)
df_h = pd.DataFrame(all_header)
df_h.head()


['[Postcode, Borough, Neighborhood\n, Canadian postal codes\n]']


,0
0,"[Postcode, Borough, Neighborhood\n, Canadian p..."


In [15]:
df_h1 = df_h[0].str.split(',', expand=True) 
df_h1.head(10)

,0,1,2,3
0,[Postcode,Borough,Neighborhood\n,Canadian postal codes\n]


In [16]:
df_h1[2] = df_h1[2].str.strip('\n]') #remove brackets from column 2
df_h1[0] = df_h1[0].str.strip('[')  #remove brackets from column 0
df_h2 = df_h1.iloc[:, 0:3] #isolate 1st 3 columns of dataframe
df_h2.head(10)

,0,1,2
0,Postcode,Borough,Neighborhood


In [17]:
#combine headers and data into one dataframe
frames = [df_h2, df2] 

df3 = pd.concat(frames)
df3.head(10)

,0,1,2
0,Postcode,Borough,Neighborhood
0,],None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Downtown Toronto,Queen's Park


In [18]:
#rename columns
df4 = df3.rename(columns=df3.iloc[0]) 
df4.head()

,Postcode,Borough,Neighborhood
0,Postcode,Borough,Neighborhood
0,],None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods


In [19]:
df5 = df4.drop(df4.index[0:1])
df5.head()

,Postcode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [20]:
list(df5.columns.values)

['Postcode', ' Borough', ' Neighborhood']

In [21]:
#remove spaces
df6 = df5.rename(columns={' Borough': 'Borough', ' Neighborhood': 'Neighborhood'})
list(df6.columns.values)


['Postcode', 'Borough', 'Neighborhood']

##### Remove cells with a borough that is Not assigned.

In [22]:
df7 = df6[df6.Borough != ' Not assigned']
df7.head(10)

,Postcode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Downtown Toronto,Queen's Park
10,M9A,Queen's Park,Not assigned
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern
14,M3B,North York,Don Mills North


In [23]:
#reset the row numbers after having droped rows
df7.reset_index(drop=True, inplace=True) 
df7.head(10)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Not assigned
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


##### Replace "not assigned" neighbourhood values with borough value:

In [24]:
for i in df7.index.values.tolist():
    if df7['Neighborhood'][i] == " Not assigned":
        df7.replace(df7['Neighborhood'][i], df7['Borough'][i], inplace= True)

df7.head(10) 

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:4278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


##### Groupby postcodes. Combine all neighourhoods within that postcode with ",":

In [25]:
df8 = df7.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(' ,'.join).reset_index()
#df8 = df7.groupby(['Postcode', 'Borough'])
df8.tail()

,Postcode,Borough,Neighborhood
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village , Martin Grove Gardens , Ri..."
101,M9V,Etobicoke,"Albion Gardens , Beaumond Heights , Humbergat..."
102,M9W,Etobicoke,Northwest


In [26]:
df8.shape

(103, 3)

# Week 3 Assignment: PART 2

In [27]:
#Method via geocoder failed to work, therefore csv file was used. 

In [28]:
df_coords = pd.read_csv("http://cocl.us/Geospatial_data")
df_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [29]:
df_merge= pd.merge(left=df8,right=df_coords, how='right', left_on='Postcode', right_on='Postal Code') #merged_left
df_merge.head()


,Postcode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge , Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [30]:
#df_final=df_merge.drop('Postal Code', axis=1)
#df_final

df_final = df_merge.drop(df_merge.columns[3], axis=1)
df_final.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge , Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [31]:
#list(df_final.columns.values)

In [32]:
#df_final =df_final.rename(columns={"Postcode":"Postal Code"}, inplace=True)
#df_final.head()

# Week 3 Assignment -  Part 3

In [ ]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

## 1. Create Toronto neighbourhood dataset to perform analysis

#### Use geopy library to get the latitude and longitude values of Toronto.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>Tor_explorer</em>, as shown below.

In [34]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="Tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a data frame for Toronto based neighbourhoods only

In [35]:
Toronto_data = df_final[df_final['Borough'].str.contains('Toronto')].reset_index(drop=True)
Toronto_data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West , India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [36]:
Toronto_data.shape

(39, 5)

#### Create a map of Toronto with neighborhoods superimposed on top.

In [37]:
# create map of Manhattan using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

#### Define Foursquare Credentials and Version

In [38]:
CLIENT_ID = 'X2PZL4TKOWMNH4F2F1ZAPJ3EQ3433Z4TJHJME55WSV3MKIO4' # your Foursquare ID
CLIENT_SECRET = 'FUM0PWZDF0OEUILMUZ5UWGE2IF3CVRJTKPXFYCTNDPMZZYFH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: X2PZL4TKOWMNH4F2F1ZAPJ3EQ3433Z4TJHJME55WSV3MKIO4
CLIENT_SECRET:FUM0PWZDF0OEUILMUZ5UWGE2IF3CVRJTKPXFYCTNDPMZZYFH


## 2. Explore Neighborhoods in Toronto

#### Create a function to get the top 50 venues in each Toronto neighbourhood within a radius of 500 meters.

In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        LIMIT = 50 # limit of number of venues returned by Foursquare API. 
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)



Toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )


 The Beaches
 The Danforth West , Riverdale
 The Beaches West , India Bazaar
 Studio District
 Lawrence Park
 Davisville North
 North Toronto West
 Davisville
 Moore Park , Summerhill East
 Deer Park , Forest Hill SE , Rathnelly , South Hill , Summerhill West
 Rosedale
 Cabbagetown , St. James Town
 Church and Wellesley
 Harbourfront
 Ryerson , Garden District
 St. James Town
 Berczy Park
 Central Bay Street
 Adelaide , King , Richmond
 Harbourfront East , Toronto Islands , Union Station
 Design Exchange , Toronto Dominion Centre
 Commerce Court , Victoria Hotel
 Roselawn
 Forest Hill North , Forest Hill West
 The Annex , North Midtown , Yorkville
 Harbord , University of Toronto
 Chinatown , Grange Park , Kensington Market
 CN Tower , Bathurst Quay , Island airport , Harbourfront West , King and Spadina , Railway Lands , South Niagara
 Stn A PO Boxes 25 The Esplanade
 First Canadian Place , Underground city
 Christie
 Dovercourt Village , Dufferin
 Little Portugal , Trinity
 Brockton 

In [40]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1205, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


Let's check how many venues were returned for each neighborhood

In [41]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide , King , Richmond",50,50,50,50,50,50
Berczy Park,50,50,50,50,50,50
"Brockton , Exhibition Place , Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
"CN Tower , Bathurst Quay , Island airport , Harbourfront West , King and Spadina , Railway Lands , South Niagara",16,16,16,16,16,16
"Cabbagetown , St. James Town",48,48,48,48,48,48
Central Bay Street,50,50,50,50,50,50
"Chinatown , Grange Park , Kensington Market",50,50,50,50,50,50
Christie,17,17,17,17,17,17


#### Let's find out how many unique categories can be curated from all the returned venues

In [42]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 217 uniques categories.


## 3. Analyze Each Toronto Neighborhood

In [43]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [44]:
Toronto_onehot.shape

(1205, 217)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [45]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,"Adelaide , King , Richmond",0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.020000,0.000000,0.00,0.000000,0.000000,0.060000,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.040000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.0000,0.000000,0.0000,0.02,0.040000,0.000000,0.000000,0.020000,0.020000,0.00,0.000000,0.060000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.060000,0.000000,0.000000,0.000000,0.00,0.02,0.000000,0.0000,0.02,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.020000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.02,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.0000,0.040000,0.000000,0.000000,0.02,0.000000,0.00,0.000000,0.020000,0.000000,0.000000,0.020000

In [46]:
Toronto_grouped.shape

(39, 217)

#### Let's print each neighborhood along with the top 5 most common venues

In [47]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Adelaide , King , Richmond----
              venue  freq
0  Asian Restaurant  0.06
1        Steakhouse  0.06
2              Café  0.06
3       Coffee Shop  0.06
4             Hotel  0.04


---- Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1        Cocktail Bar  0.06
2      Farmers Market  0.04
3  Seafood Restaurant  0.04
4         Cheese Shop  0.04


---- Brockton , Exhibition Place , Parkdale Village----
                venue  freq
0                Café  0.14
1      Breakfast Spot  0.09
2         Coffee Shop  0.09
3                 Gym  0.05
4  Italian Restaurant  0.05


---- Business Reply Mail Processing Centre 969 Eastern----
           venue  freq
0    Yoga Studio  0.06
1  Auto Workshop  0.06
2     Smoke Shop  0.06
3     Skate Park  0.06
4    Pizza Place  0.06


---- CN Tower , Bathurst Quay , Island airport , Harbourfront West , King and Spadina , Railway Lands , South Niagara----
              venue  freq
0   Airport Service  0.19
1    Airport Lou

#### Let's put that into a *pandas* dataframe

In [48]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [49]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide , King , Richmond",Steakhouse,Coffee Shop,Café,Asian Restaurant,Bar,Gastropub,Sushi Restaurant,Breakfast Spot,Hotel,Pizza Place
1,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Beer Bar,Steakhouse,Cheese Shop,Café,Farmers Market,Breakfast Spot
2,"Brockton , Exhibition Place , Parkdale Village",Café,Breakfast Spot,Coffee Shop,Grocery Store,Restaurant,Intersection,Bar,Italian Restaurant,Bakery,Climbing Gym
3,Business Reply Mail Processing Centre 969 Eas...,Yoga Studio,Spa,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Park,Restaurant
4,"CN Tower , Bathurst Quay , Island airport , H...",Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Boutique,Rental Car Location,Plane,Boat or Ferry,Harbor / Marina,Airport Food Court


## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [50]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [51]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster labels', kmeans.labels_)

Toronto_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.tail() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,M6P,West Toronto,"High Park , The Junction South",43.661608,-79.464763,0,Mexican Restaurant,Café,Thai Restaurant,Bar,Grocery Store,Bakery,Music Venue,Cajun / Creole Restaurant,Diner,Bookstore
35,M6R,West Toronto,"Parkdale , Roncesvalles",43.648960,-79.456325,0,Gift Shop,Coffee Shop,Restaurant,Dessert Shop,Bookstore,Cuban Restaurant,Movie Theater,Bar,Dog Run,Breakfast Spot
36,M6S,West Toronto,"Runnymede , Swansea",43.651571,-79.484450,0,Café,Pizza Place,Coffee Shop,Italian Restaurant,Sushi Restaurant,Bar,Falafel Restaurant,South American Restaurant,Indie Movie Theater,Smoothie Shop
37,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,0,Coffee Shop,Gym,Park,College Auditorium,Smoothie Shop,Sandwich Place,Burger Joint,Burrito Place,Café,Portuguese Restaurant
38,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eas...,43.662744,-79.321558,0,Yoga Studio,Spa,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Park,Restaurant


Finally, let's visualize the resulting clusters

In [56]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

#### Cluster 1

In [57]:
Toronto_merged.loc[Toronto_merged['Cluster labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Other Great Outdoors,Trail,Health Food Store,Pub,Wine Shop,Cuban Restaurant,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run
1,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bookstore,Grocery Store,Pub,Pizza Place,Lounge
2,East Toronto,0,Park,Pet Store,Sushi Restaurant,Movie Theater,Pub,Burrito Place,Burger Joint,Sandwich Place,Liquor Store,Brewery
3,East Toronto,0,Café,Coffee Shop,Gastropub,Brewery,Bakery,Italian Restaurant,American Restaurant,Comfort Food Restaurant,Sandwich Place,Cheese Shop
5,Central Toronto,0,Dance Studio,Park,Gym,Breakfast Spot,Clothing Store,Dog Run,Hotel,Sandwich Place,Food & Drink Shop,Wine Shop
6,Central Toronto,0,Coffee Shop,Sporting Goods Shop,Clothing Store,Yoga Studio,Mexican Restaurant,Rental Car Location,Café,Salon / Barbershop,Burger Joint,Chinese Restaurant
7,Central Toronto,0,Dessert Shop,Sandwich Place,Sushi Restaurant,Coffee Shop,Italian Restaurant,Café,Gym,Pizza Place,Costume Shop,Brewery
9,Central Toronto,0,Pub,Coffee Shop,Restaurant,Bagel Shop,Supermarket,Sushi Restaurant,Sports Bar,Athletics & Sports,Fried Chicken Joint,American Restaurant
11,Downtown Toronto,0,Coffee Shop,Bakery,Pub,Market,Pizza Place,Italian Restaurant,Flower Shop,Café,Restaurant,Caribbean Restaurant
12,Downtown Toronto,0,Sushi Restaurant,Burger Joint,Gay Bar,Gastropub,Gym,Men's Store,Coffee Shop,Restaurant,Park,Mexican Restaurant


#### Cluster 2

In [58]:
Toronto_merged.loc[Toronto_merged['Cluster labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,1,Playground,Cuban Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Diner,Dessert Shop,Department Store


#### Cluster 3

In [59]:
Toronto_merged.loc[Toronto_merged['Cluster labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,2,Garden,Wine Shop,Cuban Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Diner,Dessert Shop


#### Cluster 4

In [60]:
Toronto_merged.loc[Toronto_merged['Cluster labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,3,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Wine Shop,Cupcake Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run


#### Cluster 5

In [61]:
Toronto_merged.loc[Toronto_merged['Cluster labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,4,Park,Construction & Landscaping,Swim School,Bus Line,Wine Shop,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
10,Downtown Toronto,4,Park,Playground,Trail,Wine Shop,Cuban Restaurant,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run,Diner
